In [ ]:
import utils
from tqdm import tqdm
import os
import openai
openai.api_key = utils.get_openai_api_key()

In [ ]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["../data/sample_data/AI Career/data/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [ ]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [ ]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-base-en-v1.5", chunk_size=256
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [ ]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

# Refine

In [ ]:
from llama_index.response_synthesizers import (
    ResponseMode,
    get_response_synthesizer,
)

In [ ]:
response_synthesizer = get_response_synthesizer(
    response_mode="refine"
)

In [ ]:
query_engine = index.as_query_engine(response_synthesizer=response_synthesizer)

In [ ]:
from trulens_eval import Tru
tru = Tru()
tru.reset_database()

In [ ]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Refine Response Engine")

In [ ]:
with tru_recorder as recording:
    for question in tqdm(eval_questions, total=len(eval_questions)):
        response = query_engine.query(question)

In [ ]:
tru.get_leaderboard(app_ids=[])

# Compact

In [ ]:
response_synthesizer = get_response_synthesizer(
    response_mode=ResponseMode.COMPACT
)

In [ ]:
query_engine = index.as_query_engine(response_synthesizer=response_synthesizer)

In [ ]:
tru.reset_database()

In [ ]:
tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Compact Response Engine")

In [ ]:
with tru_recorder as recording:
    for question in tqdm(eval_questions, total=len(eval_questions)):
        response = query_engine.query(question)

In [ ]:
tru.get_leaderboard(app_ids=[])

# Simple Summary

In [ ]:
response_synthesizer = get_response_synthesizer(
    response_mode="simple_summarize"
)

In [ ]:
query_engine = index.as_query_engine(response_synthesizer=response_synthesizer)

In [ ]:
tru.reset_database()

In [ ]:
tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Simple Summary Response Engine")

In [ ]:
with tru_recorder as recording:
    for question in tqdm(eval_questions, total=len(eval_questions)):
        response = query_engine.query(question)

In [ ]:
tru.get_leaderboard(app_ids=[])